In [43]:
user_id = '579c6abd-99d0-4012-bc58-ee780164255d'  #softwareEngineer
user_id = '9448612f-8305-46e7-8306-a1e185013991'  #Backend Dev
user_id = 'c1705e9d-9950-4604-94f6-ef96177b33cc'  #ML
user_id = '9ef4d8b9-a6e5-4cef-a42f-0aa1f7d821e1'  #Product manager

In [34]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
# Load the user_certification DataFrame
user_cert_df = pd.read_csv('user_certification.csv')

# Assuming you have another DataFrame for certifications that maps certification IDs to names
certifications_df = pd.read_csv('certifications.csv')  # Contains 'certificationId' and 'certificationName'

# Merge the user_cert_df with the certifications_df to get certification names
user_cert_df = user_cert_df.merge(certifications_df, on='_id', how='left')

# Pivot the data to create a user-certification matrix
user_cert_matrix = user_cert_df.pivot_table(index='userId_x', columns='certificationId', values='isVerified').fillna(0)

# Compute cosine similarity (or use previously defined method)
user_similarity = cosine_similarity(user_cert_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_cert_matrix.index, columns=user_cert_matrix.index)

# Function to recommend certifications for a specific user based on similar users
def recommend_certifications(user_id, num_recommendations=5):
    # Ensure user_id exists in user_cert_matrix
    if user_id not in user_cert_matrix.index:
        return f"User ID {user_id} not found in the dataset."
    
    # Get similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]
    
    # Aggregate certifications from similar users
    similar_users_certifications = user_cert_matrix.loc[similar_users].sum().sort_values(ascending=False)
    
    # Filter out certifications the user has already completed
    user_certifications = user_cert_matrix.loc[user_id]
    recommended_certs = similar_users_certifications[user_certifications == 0].head(num_recommendations)
    
    # Get certification names and user roles
    recommendations = []
    for cert_id in recommended_certs.index:
        # Get the certification name
        cert_name = certifications_df.loc[certifications_df['_id'] == cert_id, 'name'].values[0]
        
        # Get user roles for each similar user
        for user in similar_users:
            user_role = user_cert_df.loc[user_cert_df['userId_x'] == user, 'role_id'].values[0]  # Adjust 'role_id' if needed
            recommendations.append({
                'user_id': user,
                'certification_id': cert_id,
                'certification_name': cert_name,
                'user_role': user_role
            })
    
    return pd.DataFrame(recommendations)

recommended_certifications = recommend_certifications(user_id)
recommended_certifications


                                     user_id  \
0       5eb97b40-b5a1-4ae7-82be-12b7f3380f59   
1       fab59c14-762a-4b4c-a839-1a85f80d5712   
2       4aaf15c0-460a-41f9-baaf-1ca60fd55a54   
3       c8351dcd-8059-4aa0-8e78-926b02d76110   
4       06a7c716-5ecf-46b5-bc09-c2518986c344   
...                                      ...   
149990  57199b78-7b8f-4014-b7a7-f19736ed4d04   
149991  5714c2b6-a552-4dfd-93f7-a4e6ee19705d   
149992  5713efe5-c292-4dee-b652-3c05ff63df82   
149993  57133aae-296a-430c-a54e-4ed1462dab56   
149994  ffff9f64-e7bb-49a7-ba11-ecd5b30c1695   

                            certification_id  \
0       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
1       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
2       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
3       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
4       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
...                                      ...   
149990  34ca4f75-b2c9-455c-920b-a0d9597f3259   
149991  34ca4f75-b2c9-455c-920b-a0d9597

In [45]:
def recommend_certifications(user_id, num_recommendations=5):
    # Ensure user_id exists in user_cert_matrix
    if user_id not in user_cert_matrix.index:
        return f"User ID {user_id} not found in the dataset."
    
    # Get similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]
    
    # Aggregate certifications from similar users
    similar_users_certifications = user_cert_matrix.loc[similar_users].sum().sort_values(ascending=False)
    
    # Filter out certifications the user has already completed
    user_certifications = user_cert_matrix.loc[user_id]
    recommended_certs = similar_users_certifications[user_certifications == 0].head(num_recommendations)
    
    # Pre-fetch certification names and user roles to avoid repeated lookups
    cert_name_map = dict(zip(certifications_df['_id'], certifications_df['name']))
    
    # Extract roles for all similar users at once
    user_roles = user_cert_df.set_index('userId_x')['role_id'].loc[similar_users].to_dict()
    
    # Create a DataFrame for the recommendations
    recommendations = [
        {
            'user_id': similar_user,
            'certification_id': cert_id,
            'certification_name': cert_name_map.get(cert_id, None),
            'user_role': user_roles.get(similar_user, None)
        }
        for cert_id in recommended_certs.index
        for similar_user in similar_users if cert_id in user_cert_matrix.loc[similar_user].index[user_cert_matrix.loc[similar_user] == 0]
    ]
    
    return pd.DataFrame(recommendations)

# Example usage
recommended_certifications = recommend_certifications(user_id)
print(recommended_certifications)


                                     user_id  \
0       5eb97b40-b5a1-4ae7-82be-12b7f3380f59   
1       fab59c14-762a-4b4c-a839-1a85f80d5712   
2       4aaf15c0-460a-41f9-baaf-1ca60fd55a54   
3       c8351dcd-8059-4aa0-8e78-926b02d76110   
4       06a7c716-5ecf-46b5-bc09-c2518986c344   
...                                      ...   
147519  57199b78-7b8f-4014-b7a7-f19736ed4d04   
147520  5714c2b6-a552-4dfd-93f7-a4e6ee19705d   
147521  5713efe5-c292-4dee-b652-3c05ff63df82   
147522  57133aae-296a-430c-a54e-4ed1462dab56   
147523  ffff9f64-e7bb-49a7-ba11-ecd5b30c1695   

                            certification_id  \
0       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
1       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
2       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
3       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
4       58ac6d2f-bb0d-4e4e-bafa-18496c2ed57d   
...                                      ...   
147519  34ca4f75-b2c9-455c-920b-a0d9597f3259   
147520  34ca4f75-b2c9-455c-920b-a0d9597

Content based

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd

# Load DataFrames
user_skill_df = pd.read_csv('user_skill.csv')
role_skill_df = pd.read_csv('role_skill.csv')

# Print the columns to check for 'certificationId'
print(role_skill_df.columns)

# Assuming 'certificationId' exists, merge data to get skills related to each certification (from role_skill)
# Adjust the column name if necessary
certification_skills = role_skill_df[['certificationId', 'skill_name']].groupby('certificationId')['skill_name'].apply(lambda x: ' '.join(x))

# Calculate TF-IDF for skills associated with certifications
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(certification_skills)

# Function to recommend certifications based on user skills
def recommend_content_based(user_id, num_recommendations=5):
    # Get the user's skills
    user_skills = ' '.join(user_skill_df[user_skill_df['userId_x'] == user_id]['skill_name'])
    user_tfidf = tfidf.transform([user_skills])
    
    # Calculate cosine similarity
    cosine_sim = linear_kernel(user_tfidf, tfidf_matrix).flatten()
    certification_indices = cosine_sim.argsort()[-num_recommendations:][::-1]
    
    # Return recommended certifications
    return certification_skills.index[certification_indices]


recommended_certifications = recommend_content_based(user_id)
print(recommended_certifications)


KeyError: "['certificationId'] not in index"

KNN

In [21]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Load the user_certification DataFrame (assuming it has role information)
user_cert_df = pd.read_csv('user_certification.csv')

# Pivot the data to create a user-certification matrix
user_cert_matrix = user_cert_df.pivot_table(index='userId_x', columns='certificationId', values='isVerified').fillna(0)

# Create a DataFrame to map user IDs to names, roles, and certification IDs to names
user_info = user_cert_df[['userId_x', 'user_name', 'role_id', 'role_name']].drop_duplicates().set_index('userId_x')
cert_names = user_cert_df[['certificationId', 'certificationName']].drop_duplicates().set_index('certificationId')

# Calculate the number of unique roles
unique_roles = user_cert_df['role_id'].nunique()  # or 'role_name' if you want to use role names
print(f"Number of unique roles: {unique_roles}")

# Build the KNN model with neighbors based on the unique roles
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=unique_roles)
knn_model.fit(user_cert_matrix)

# Function to recommend top 5 certifications using KNN with neighbor count based on roles
def recommend_knn_by_role(user_id):
    # Ensure that the user exists
    if user_id not in user_cert_matrix.index:
        return f"User ID {user_id} not found in the dataset."
    
    # Get the index of the user in the matrix
    user_idx = user_cert_matrix.index.get_loc(user_id)

    # Find nearest neighbors (based on unique role count)
    distances, indices = knn_model.kneighbors([user_cert_matrix.iloc[user_idx]], n_neighbors=unique_roles + 1)
    
    # Ignore the first neighbor, as it will be the user itself
    similar_users = user_cert_matrix.index[indices.flatten()[1:]]
    
    # Aggregate certifications from similar users and recommend
    similar_users_certifications = user_cert_matrix.loc[similar_users].sum().sort_values(ascending=False)
    user_certifications = user_cert_matrix.loc[user_id]
    
    # Filter out certifications that the user already has
    recommended_certs = similar_users_certifications[user_certifications == 0].head(5)  # Get top 5

    # Create a DataFrame for the recommended certifications with user names
    recommended_cert_names = pd.DataFrame(recommended_certs).reset_index()
    recommended_cert_names.columns = ['certificationId', 'count']
    recommended_cert_names = recommended_cert_names.merge(cert_names, on='certificationId')

    # Get user info for the requesting user
    user_name = user_info.loc[user_id, 'user_name']
    user_role = user_info.loc[user_id, 'role_name']  # Get the user's role name

    # Displaying the user name and role along with recommended certifications
    print(f"Top 5 Recommendations for {user_name} (Role: {user_role}):\n")
    
    for _, row in recommended_cert_names.iterrows():
        print(f"- {row['certificationName']}")

    # return recommended_cert_names

recommend_knn_by_role(user_id)


Number of unique roles: 15
Top 5 Recommendations for Mr. Sean Smith (Role: Product Manager):

- Certified Agile Leadership (CAL)
- Certified Information Systems Security Professional
- Certified Technical Writer (CTW)
- Databricks Certified Associate Developer for Apache Spark
- CompTIA Security+


c:\Users\RithikHarendarMahesh\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
